### AHP Function

In [1]:
import numpy as np
import pandas as pd


# Function untuk menghitung bobot (weights)
def ahp_weights(matrix_comparism):
    
    # matrix_comparism = np.array(matrix_comparism)
    n = matrix_comparism.shape[0]

    # Menghitung nilai geometris dari matriks perbandingan
    M_I = np.prod(matrix_comparism, axis=1)
    W_bar_I = np.power(M_I, 1/n)

    # Normalisasi bobot relatif agar jumlahnya 1
    W_I = W_bar_I / np.sum(W_bar_I)
    
    return W_I, W_bar_I

Dict_RI = {
    1: 0.00, 
    2: 0.00, 
    3: 0.58, 
    4: 0.90, 
    5: 1.12, 
    6: 1.24, 
    7: 1.32, 
    8: 1.41, 
    9: 1.45,
    10: 1.49,
    11: 1.51,
    12: 1.48,
    13: 1.56,
    14: 1.57,
    15: 1.58,
}

# Function untuk menghitung konsistensi
def consistency(matrix_comparism, W_I):
    n = matrix_comparism.shape[0]

    # Menghitung lambda max
    AW= np.dot(matrix_comparism, W_I)
    lambda_max = np.sum(AW / (n*W_I))
    
    # Menghitung CI
    CI = (lambda_max - n) / (n - 1)

    # Mengambil RI sesuai dengan n
    RI = Dict_RI[n]

    # Menghitung CR
    CR = CI / RI 

    return lambda_max, CI, CR, RI, n

def ahp_rumus(matrix_comparism, labels):
    # Hitung bobot
    W_I, W_bar_I = ahp_weights(matrix_comparism)
    
    # Hitung konsistensi
    lambda_max, CI, CR, RI, n = consistency(matrix_comparism, W_I)
    
    # Buat DataFrame untuk menampilkan hasil matriks perbandingan, geometris, dan bobot
    df = pd.DataFrame(matrix_comparism, 
                      index=labels, 
                      columns=labels,
                    )
    df['Geometris'] = W_bar_I
    df['Weight'] = W_I
    
    # Cetak output sesuai format
 
    print("Comparism Matrix")
    print(df.round(3))
    print(f'\nJumlah Bobot = {round(W_I.sum())}')
    print(f'\nLambda_max= {lambda_max:.4f}')
    print(f'RI= {n} : {RI}')
    print(f'CI= {CI:.4f}')
    print(f'CR= {CR:.4}')

    # Evaluasi konsistensi
    if CR < 0.1:
        print("\nBobot konsisten")
    else:
        print("Bobot Tidak Konsisten")

    return W_I

Penggunaan AHP

In [2]:

iklim_topografi = 3
iklim_tanah = 1
topografi_tanah = 1/5

matrix_comparism = np.array([
    [1, iklim_topografi, iklim_tanah],
    [1/iklim_topografi, 1, topografi_tanah],
    [1/iklim_tanah, 1/topografi_tanah, 1]
])

# Label untuk parameter
labels = ['iklim', 'topografi', 'tanah']

# Memanggil fungsi AHP untuk menampilkan hasil
# ahp_rumus(matrix_comparism, labels)

# Memanggil fungsi AHP dan mendapatkan hasil bobot (W_I)
W_I = ahp_rumus(matrix_comparism, labels)

# Membuat DataFrame hanya dengan label dan bobot
df_output = pd.DataFrame({
    'Label': labels,
    'Weight': W_I
})

# Menyimpan DataFrame ke file Excel dengan nama yang berbeda
# df_output.to_excel('./dataset/bobot_ahp/parameter_utama.xlsx', index=False)


Comparism Matrix
           iklim  topografi  tanah  Geometris  Weight
iklim      1.000        3.0    1.0      1.442   0.405
topografi  0.333        1.0    0.2      0.405   0.114
tanah      1.000        5.0    1.0      1.710   0.481

Jumlah Bobot = 1

Lambda_max= 3.0291
RI= 3 : 0.58
CI= 0.0145
CR= 0.02505

Bobot konsisten


### PROFILE MATCHING

Interpolasi Iklim

In [45]:
import pandas as pd

# Load dataset varietas (Profile Ideal)
df_varietas = pd.read_excel('./dataset/split_data/varietas_iklim.xlsx')

# Load dataset lahan (Profile Alternatif)
df_lahan = pd.read_excel('./dataset/split_data/lahan_iklim.xlsx')

# Function utama untuk interpolasi
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Function untuk menghitung interpolasi berdasarkan varietas tertentu (misalnya, hanya V1)
def hitung_skor_varietas_sederhana(row, jenis_varietas):
    # Memastikan jenis varietas ada di dataset
    if jenis_varietas not in df_varietas['jenis_varietas'].values:
        raise ValueError(f"Varietas {jenis_varietas} tidak ditemukan di dataset.")

    # Mengambil data varietas berdasarkan jenis_varietas
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]
    
    # Menghitung skor interpolasi untuk masing-masing parameter
    hasil = {
        'suhu': interpolasi(row['suhu_mean'], var_data['suhu_min'], var_data['suhu_max']),
        'curah_hujan': interpolasi(row['curah_hujan'], var_data['curah_hujan_min'], var_data['curah_hujan_max']),
        'kelembapan_udara': interpolasi(row['kelembapan_udara'], var_data['kelembapan_udara_min'], var_data['kelembapan_udara_max']),
        'penyinaran': interpolasi(row['penyinaran'], var_data['penyinaran_min'], var_data['penyinaran_max'])
    }
    
    return hasil

#Penggunaan 
def hitung_skor(row):
    return hitung_skor_varietas_sederhana(row, 'V1')

lahan_scores = df_lahan.apply(hitung_skor, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi 
interpolasi_iklim = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

print("Hasil Skor Interpolasi Iklim untuk Varietas V1:")
interpolasi_iklim


Hasil Skor Interpolasi Iklim untuk Varietas V1:


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran
0,Bakung,5.0,5.0000,5.0,5.0
1,Binangun,5.0,5.0000,5.0,5.0
2,Doko,5.0,1.0000,5.0,5.0
3,Gandusari,5.0,1.0000,5.0,5.0
4,Garum,5.0,1.0000,5.0,5.0
5,Kademangan,5.0,3.5296,5.0,5.0
6,Kanigoro,5.0,5.0000,5.0,5.0
7,Kesamben,5.0,1.9368,5.0,5.0
8,Nglegok,5.0,1.0000,5.0,5.0
9,Panggungrejo,5.0,5.0000,5.0,5.0


Interpolasi topografi

In [47]:
import pandas as pd

df_varietas = pd.read_excel('./dataset/split_data/varietas_topografi.xlsx')

df_lahan = pd.read_excel('./dataset/split_data/lahan_topografi.xlsx')

def interpolasi_ketinggian(x, min_val,max_val):
    if  min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val + max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

def interpolasi_kemiringan(x,min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif max_val < x <= 80 :
        return 5 + ((x - max_val) / (80  - max_val)) * (1 - 5)
    else: 
        return 1

def interpolasi_drainase(x, value):
    if x == value:
        return 5
    else:
        return 1
    
def hitung_topografi(row, jenis_varietas):
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]

    hasil = {
        'ketinggian': interpolasi_ketinggian(row['ketinggian'], var_data['ketinggian_min'], var_data['ketinggian_max']),
        'kemiringan' : interpolasi_kemiringan(row['kemiringan'], var_data['kemiringan_min'], var_data['kemiringan_max']),
        'drainase': interpolasi_drainase(row['drainase'], var_data['drainase'])
    }

    return hasil

#Penggunaan 
def hitung_skor(row):
    return hitung_topografi(row, 'V1')

lahan_scores = df_lahan.apply(hitung_skor, axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi 
interpolasi_topo = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

print("Hasil Skor Interpolasi Topografi untuk Varietas V1:")
interpolasi_topo



Hasil Skor Interpolasi Topografi untuk Varietas V1:


,nama_lahan,ketinggian,kemiringan,drainase
0,Bakung,2.280,5.000000,5.0
1,Binangun,2.944,5.000000,5.0
2,Doko,3.792,5.000000,5.0
3,Gandusari,3.784,5.000000,5.0
4,Garum,3.208,5.000000,5.0
5,Kademangan,2.968,5.000000,1.0
6,Kanigoro,2.464,5.000000,5.0
7,Kesamben,2.776,5.000000,5.0
8,Nglegok,3.600,5.000000,5.0
9,Panggungrejo,3.480,3.857143,5.0


Interpolasi Tanah

In [48]:
import pandas as pd

# Membaca dataset
df_varietas = pd.read_excel('./dataset/split_data/varietas_tanah.xlsx')
df_lahan = pd.read_excel('./dataset/split_data/lahan_tanah.xlsx')
df_jenis_tanah = pd.read_excel('./dataset/bobot_ahp/bobot_jenis_tanah.xlsx')

# Fungsi interpolasi untuk variabel numerik
def interpolasi(x, min_val, max_val):
    if min_val <= x <= max_val:
        return 5
    elif 0 <= x < min_val:
        return 1 + ((x - 0) / (min_val - 0)) * (5 - 1)
    elif max_val < x <= (min_val+max_val):
        return 5 + ((x - max_val) / ((min_val + max_val) - max_val)) * (1 - 5)
    else:
        return 1

# Interpolasi untuk tekstur
def interpolasi_tekstur(x, value):
    if x in ['lempung', 'liat', 'berpasir', 'lempung berpasir']:
        return 5
    else:
        return 1

def interpolasi_jenis_tanah(x):
    if -1 <= x <= 0:
        return 5 + ((x - 0) / (-1 - 0)) * (4 - 5)
    elif 0 <= x <= 1:
        return 5 + ((x - 0) / (1 - 0)) * (4.5 - 5)

# Mengubah jenis tanah menjadi bobot berdasarkan data bobot jenis tanah
def ubah_jenis_tanah_ke_bobot(df_lahan, df_bobot_tanah):
    bobot_dict = pd.Series(df_bobot_tanah.Weight.values, index=df_bobot_tanah.Label).to_dict()
    df_lahan['jenis_tanah_bobot'] = df_lahan['jenis_tanah'].map(bobot_dict)
    return df_lahan

df_lahan = ubah_jenis_tanah_ke_bobot(df_lahan, df_jenis_tanah)

# Ubah varietas tanah ke bobot
def ubah_varietas_tanah_ke_bobot(df_varietas_tanah, df_bobot_tanah):
    bobot_dict = pd.Series(df_bobot_tanah.Weight.values, index=df_bobot_tanah.Label).to_dict()
    df_varietas_tanah['jenis_tanah_bobot'] = df_varietas_tanah['jenis_tanah'].map(bobot_dict)
    return df_varietas_tanah

df_varietas = ubah_varietas_tanah_ke_bobot(df_varietas, df_jenis_tanah)

# Hitung selisih bobot antara lahan dan varietas
def hitung_selisih_bobot(df_lahan, df_varietas, varietas):
    varietas_bobot = df_varietas[df_varietas['jenis_varietas'] == varietas].iloc[0]['jenis_tanah_bobot']
    df_lahan['selisih_bobot'] = df_lahan['jenis_tanah_bobot'] - varietas_bobot
    return df_lahan


# Hitung skor tanah dengan menggunakan data selisih bobot
def hitung_skor_tanah(row, jenis_varietas):
    var_data = df_varietas[df_varietas['jenis_varietas'] == jenis_varietas].iloc[0]
    
    hasil = {
        'ph': interpolasi(row['ph'], var_data['ph_min'], var_data['ph_max']),
        'kelembapan_tanah': interpolasi(row['kelembapan_tanah'], var_data['kelembapan_tanah_min'], var_data['kelembapan_tanah_max']),
        'jenis_tanah': interpolasi_jenis_tanah(row['selisih_bobot']),
        'tekstur_tanah': interpolasi_tekstur(row['tekstur_tanah'], var_data['tekstur_tanah'])
    }
    return hasil

# Penggunaan
df_lahan = hitung_selisih_bobot(df_lahan, df_varietas, 'V1')
lahan_scores = df_lahan.apply(lambda row: hitung_skor_tanah(row, 'V1'), axis=1)

# Menampilkan DataFrame hasil dengan nama lahan dan skor interpolasi 
interpolasi_tanah = pd.concat([df_lahan['nama_lahan'], lahan_scores.apply(pd.Series)], axis=1)

print("Hasil Skor Interpolasi Tanah untuk Varietas V1:")
interpolasi_tanah


Hasil Skor Interpolasi Tanah untuk Varietas V1:


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah
0,Bakung,5.0,4.626667,4.765187,1.0
1,Binangun,5.0,4.626667,4.658762,1.0
2,Doko,5.0,3.640000,5.000000,5.0
3,Gandusari,5.0,3.640000,4.880826,5.0
4,Garum,5.0,3.640000,4.716732,5.0
5,Kademangan,5.0,4.160000,4.765187,5.0
6,Kanigoro,5.0,4.626667,4.716732,5.0
7,Kesamben,5.0,4.986667,4.658762,5.0
8,Nglegok,5.0,3.640000,4.880826,5.0
9,Panggungrejo,5.0,4.626667,4.765187,1.0


## Nilai Kriteria

Nilai Kriteria Iklim

In [49]:
#bobot ahp sub parameter iklim
bobot_iklim = pd.read_excel('./dataset/bobot_ahp/parameter_iklim.xlsx')
bobot_iklim_dict = bobot_iklim.set_index('Label')['Weight'].to_dict()
bobot_iklim_dict

{'suhu': 0.2117957615233342,
 'curah_hujan': 0.5251582768815123,
 'kelembapan_udara': 0.05125020007181938,
 'penyinaran': 0.2117957615233342}

In [50]:
nk_iklim = interpolasi_iklim.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']:
    nk_iklim[kolom] = nk_iklim[kolom] * bobot_iklim_dict[kolom]

nk_iklim['total_skor'] = nk_iklim[['suhu', 'curah_hujan', 'kelembapan_udara', 'penyinaran']].sum(axis=1)

nk_iklim


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000


nilai kriteria topografi

In [51]:
bobot_topo = pd.read_excel('./dataset/bobot_ahp/parameter_topografi.xlsx')
bobot_topo_dict = bobot_topo.set_index('Label')['Weight'].to_dict()
bobot_topo_dict

{'ketinggian': 0.4806400851812919,
 'kemiringan': 0.1139723667233914,
 'drainase': 0.4053875480953168}

In [52]:
nk_topografi = interpolasi_topo.copy()

for kolom in ['ketinggian','kemiringan','drainase']:
    nk_topografi[kolom] = nk_topografi[kolom] * bobot_topo_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
nk_topografi['total_skor'] = nk_topografi[['ketinggian','kemiringan','drainase']].sum(axis=1)

nk_topografi

,nama_lahan,ketinggian,kemiringan,drainase,total_skor
0,Bakung,1.095859,0.569862,2.026938,3.692659
1,Binangun,1.415004,0.569862,2.026938,4.011804
2,Doko,1.822587,0.569862,2.026938,4.419387
3,Gandusari,1.818742,0.569862,2.026938,4.415542
4,Garum,1.541893,0.569862,2.026938,4.138693
5,Kademangan,1.426540,0.569862,0.405388,2.401789
6,Kanigoro,1.184297,0.569862,2.026938,3.781097
7,Kesamben,1.334257,0.569862,2.026938,3.931056
8,Nglegok,1.730304,0.569862,2.026938,4.327104
9,Panggungrejo,1.672627,0.439608,2.026938,4.139173


Nilai Kriteria Tanah

In [36]:
#bobot ahp sub parameter tanah
bobot_tanah = pd.read_excel('./dataset/bobot_ahp/parameter_tanah.xlsx')
bobot_tanah_dict = bobot_tanah.set_index('Label')['Weight'].to_dict()
bobot_tanah_dict

{'ph': 0.3814374052035476,
 'kelembapan_tanah': 0.09848671453159402,
 'jenis_tanah': 0.4333965668873818,
 'tekstur_tanah': 0.08667931337747636}

In [41]:
# Variabel untuk menyimpan hasil perkalian dengan bobot
nk_tanah = interpolasi_tanah.copy()

# Melakukan perkalian setiap kolom dengan bobot yang sesuai
for kolom in ['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']:
    nk_tanah[kolom] = nk_tanah[kolom] * bobot_tanah_dict[kolom]

# Menambahkan kolom 'total_skor' yang berisi jumlah dari kolom yang telah dikalikan bobot
nk_tanah['total_skor'] = nk_tanah[['ph', 'kelembapan_tanah', 'jenis_tanah', 'tekstur_tanah']].sum(axis=1)

nk_tanah


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor
0,Bakung,1.907187,0.455665,2.065215,0.086679,4.514747
1,Binangun,1.907187,0.455665,2.019091,0.086679,4.468623
2,Doko,1.907187,0.358492,2.166983,0.433397,4.866058
3,Gandusari,1.907187,0.358492,2.115333,0.433397,4.814409
4,Garum,1.907187,0.358492,2.044216,0.433397,4.743291
5,Kademangan,1.907187,0.409705,2.065215,0.433397,4.815504
6,Kanigoro,1.907187,0.455665,2.044216,0.433397,4.840464
7,Kesamben,1.907187,0.491120,2.019091,0.433397,4.850795
8,Nglegok,1.907187,0.358492,2.115333,0.433397,4.814409
9,Panggungrejo,1.907187,0.455665,2.065215,0.086679,4.514747


### Rangking

In [54]:
#bobot parameter utama
bobot_utama = pd.read_excel('./dataset/bobot_ahp/parameter_utama.xlsx')
bobot_utama_dict = bobot_utama.set_index('Label')['Weight'].to_dict()
bobot_utama_dict

{'iklim': 0.4053875480953168,
 'topografi': 0.1139723667233914,
 'tanah': 0.4806400851812919}

In [57]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_iklim = nk_iklim.copy()
total_skor_iklim['total_skor_iklim'] = total_skor_iklim['total_skor'] * bobot_utama_dict['iklim']
total_skor_iklim


,nama_lahan,suhu,curah_hujan,kelembapan_udara,penyinaran,total_skor,total_skor_iklim
0,Bakung,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
1,Binangun,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
2,Doko,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
3,Gandusari,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
4,Garum,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
5,Kademangan,1.058979,1.853599,0.256251,1.058979,4.227807,1.713900
6,Kanigoro,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938
7,Kesamben,1.058979,1.017127,0.256251,1.058979,3.391335,1.374805
8,Nglegok,1.058979,0.525158,0.256251,1.058979,2.899367,1.175367
9,Panggungrejo,1.058979,2.625791,0.256251,1.058979,5.000000,2.026938


In [59]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_topo = nk_topografi.copy()
total_skor_topo['total_skor_topo'] = total_skor_topo['total_skor'] * bobot_utama_dict['topografi']
total_skor_topo


,nama_lahan,ketinggian,kemiringan,drainase,total_skor,total_skor_topo
0,Bakung,1.095859,0.569862,2.026938,3.692659,0.420861
1,Binangun,1.415004,0.569862,2.026938,4.011804,0.457235
2,Doko,1.822587,0.569862,2.026938,4.419387,0.503688
3,Gandusari,1.818742,0.569862,2.026938,4.415542,0.503250
4,Garum,1.541893,0.569862,2.026938,4.138693,0.471697
5,Kademangan,1.426540,0.569862,0.405388,2.401789,0.273738
6,Kanigoro,1.184297,0.569862,2.026938,3.781097,0.430941
7,Kesamben,1.334257,0.569862,2.026938,3.931056,0.448032
8,Nglegok,1.730304,0.569862,2.026938,4.327104,0.493170
9,Panggungrejo,1.672627,0.439608,2.026938,4.139173,0.471751


In [56]:
# Mengalikan kolom 'total_skor' dengan bobot 'iklim'
total_skor_tanah = nk_tanah.copy()
total_skor_tanah['total_skor_tanah'] = total_skor_tanah['total_skor'] * bobot_utama_dict['tanah']
total_skor_tanah


,nama_lahan,ph,kelembapan_tanah,jenis_tanah,tekstur_tanah,total_skor,total_skor_tanah
0,Bakung,1.907187,0.455665,2.065215,0.086679,4.514747,2.169968
1,Binangun,1.907187,0.455665,2.019091,0.086679,4.468623,2.147799
2,Doko,1.907187,0.358492,2.166983,0.433397,4.866058,2.338823
3,Gandusari,1.907187,0.358492,2.115333,0.433397,4.814409,2.313998
4,Garum,1.907187,0.358492,2.044216,0.433397,4.743291,2.279816
5,Kademangan,1.907187,0.409705,2.065215,0.433397,4.815504,2.314524
6,Kanigoro,1.907187,0.455665,2.044216,0.433397,4.840464,2.326521
7,Kesamben,1.907187,0.491120,2.019091,0.433397,4.850795,2.331487
8,Nglegok,1.907187,0.358492,2.115333,0.433397,4.814409,2.313998
9,Panggungrejo,1.907187,0.455665,2.065215,0.086679,4.514747,2.169968


perangkingan

In [63]:
# Menggabungkan ketiga DataFrame berdasarkan 'nama_lahan'
combined_scores = total_skor_iklim.merge(total_skor_tanah, on='nama_lahan').merge(total_skor_topo, on='nama_lahan')

# Menambahkan kolom total skor dari penjumlahan 'total_skor_iklim', 'total_skor_tanah', dan 'total_skor_topo'
combined_scores['total_skor'] = (
    combined_scores['total_skor_iklim'] + 
    combined_scores['total_skor_tanah'] + 
    combined_scores['total_skor_topo']
)

# Menghitung peringkat dan mengubahnya menjadi integer
combined_scores['rank'] = combined_scores['total_skor'].rank(ascending=False, method='min').astype(int)

# Mengurutkan berdasarkan 'rank'
sorted_scores = combined_scores.sort_values(by='rank')

# Menampilkan hasil yang diurutkan
rangking = sorted_scores[['nama_lahan', 'total_skor', 'rank']]
rangking


,nama_lahan,total_skor,rank
11,Sanankulon,4.831290,1
12,Selopuro,4.788344,2
6,Kanigoro,4.784399,3
15,Sutojayan,4.769938,4
9,Panggungrejo,4.668657,5
1,Binangun,4.631972,6
0,Bakung,4.617767,7
13,Selorejo,4.490661,8
18,Wates,4.401840,9
5,Kademangan,4.302162,10
